<a href="https://colab.research.google.com/github/Tabukag/jukebox-ai/blob/master/Jukebox_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install git+https://github.com/openai/jukebox.git

  Cloning https://github.com/openai/jukebox.git to /tmp/pip-req-build-tusiv72h
  Running command git clone -q https://github.com/openai/jukebox.git /tmp/pip-req-build-tusiv72h
  Created wheel for jukebox: filename=jukebox-1.0-cp36-none-any.whl size=197563 sha256=c2e35757c810811a378143ee9c14830b3d66c65c917469c482c55b146e771cf8
  Stored in directory: /tmp/pip-ephem-wheel-cache-d16apgd1/wheels/bd/b6/f9/ad38a67dd989a522bbe6677e95efbc4607cdcf71e7249485fe
Successfully built jukebox


In [2]:
import jukebox
import torch as t
import librosa
import os
from IPython.display import Audio
from jukebox.make_models import make_vqvae, make_prior, MODELS, make_model
from jukebox.hparams import Hyperparams, setup_hparams
from jukebox.sample import sample_single_window, _sample, \
                           sample_partial_window, upsample
from jukebox.utils.dist_utils import setup_dist_from_mpi
from jukebox.utils.torch_utils import empty_cache
rank, local_rank, device = setup_dist_from_mpi()

Using cuda True


In [3]:
model = "5b_lyrics" # or "1b_lyrics"     
hps = Hyperparams()
hps.sr = 44100
hps.n_samples = 3 if model=='5b_lyrics' else 8
hps.name = 'samples'
chunk_size = 16 if model=="5b_lyrics" else 32
max_batch_size = 3 if model=="5b_lyrics" else 16
hps.levels = 3
hps.hop_fraction = [.5,.5,.125]

vqvae, *priors = MODELS[model]
vqvae = make_vqvae(setup_hparams(vqvae, dict(sample_length = 1048576)), device)
top_prior = make_prior(setup_hparams(priors[-1], dict()), vqvae, device)


Restored from /root/.cache/jukebox-assets/models/5b/vqvae.pth.tar
0: Loading vqvae in eval mode
Loading artist IDs from /usr/local/lib/python3.6/dist-packages/jukebox/data/ids/v2_artist_ids.txt
Loading artist IDs from /usr/local/lib/python3.6/dist-packages/jukebox/data/ids/v2_genre_ids.txt
Level:2, Cond downsample:None, Raw to tokens:128, Sample length:1048576
0: Converting to fp16 params
Restored from /root/.cache/jukebox-assets/models/5b_lyrics/prior_level_2.pth.tar
0: Loading prior in eval mode


In [0]:
sample_length_in_seconds = 157

In [0]:
hps.sample_length = (int(sample_length_in_seconds*hps.sr)//top_prior.raw_to_tokens)*top_prior.raw_to_tokens
assert hps.sample_length >= top_prior.n_ctx*top_prior.raw_to_tokens, f'Please choose a larger sampling rate'

In [0]:
metas = [dict(artist = "The Weeknd",
            genre = "R&B",
            total_length = hps.sample_length,
            offset = 0,
            lyrics = """
gotta keep it going for the promo 
said to keep our kisses for the low though
Im just tryna get out of the friendzone
cause you look even better than your photos
gotta find your house, send me the info
driving through the gated residential
you love I was coming, sent your friends home 
if you wanna hide let your friends know 

I only want you when it's past the night 
the only time when it feels right
i only want you to hold me not feel me 
Keep it fucked up for the real me
when im fucked up thats the real me yeah
I only want you when it's past the night 
the only time when it feels right
i only want you to hold me not feel me 
Keep it fucked up for the real me
when im fucked up can't you see me babe

When you gonna tell me keep it simple
I only keep it up to keep it simple
I fucked two more bitches just to warn you
never gonna slow this fucking tempo
always wanna send me off to rehab
when im just living for this fucking moment
and you really want a fucking real love

I only want you when it's past the night 
the only time when it feels right
i only want you to hold me not feel me 
Keep it fucked up for the real me
when im fucked up thats the real me yeah
I only want you when it's past the night 
the only time when it feels right
i only want you to hold me not feel me 
Keep it fucked up for the real me
when im fucked up can't you see me babe

            """,
            ),
          ] * hps.n_samples
labels = [None, None, top_prior.labeller.get_batch_labels(metas, 'cuda')]

In [0]:
sampling_temperature = .98

lower_batch_size = 16
max_batch_size = 3 if model == "5b_lyrics" else 16
lower_level_chunk_size = 32
chunk_size = 16 if model == "5b_lyrics" else 32
sampling_kwargs = [dict(temp=.99, fp16=True, max_batch_size=lower_batch_size,
                        chunk_size=lower_level_chunk_size),
                    dict(temp=0.99, fp16=True, max_batch_size=lower_batch_size,
                         chunk_size=lower_level_chunk_size),
                    dict(temp=sampling_temperature, fp16=True, 
                         max_batch_size=max_batch_size, chunk_size=chunk_size)]

In [0]:
zs = [t.zeros(hps.n_samples,0,dtype=t.long, device='cuda') for _ in range(len(priors))]
zs = _sample(zs, labels, sampling_kwargs, [None, None, top_prior], [2], hps)

In [0]:
Audio(f'{hps.name}/level_2/item_0.wav')

In [0]:
if True:
  del top_prior
  empty_cache()
  top_prior=None
upsamplers = [make_prior(setup_hparams(prior, dict()), vqvae, 'cpu') for prior in priors[:-1]]
labels[:2] = [prior.labeller.get_batch_labels(metas, 'cuda') for prior in upsamplers]

In [0]:
ls {checkpoint_dir}